In [33]:
import pandas as pd
import numpy as np 
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from torchsummary import summary
from torch.autograd import Variable

# Load data

In [34]:
df = pd.read_csv(r'..\Dataset\SWAT\SWaT.csv') 
df.insert(78,'Label','Normal')
#Attack on FIT401
df.loc[((df['Timestamp'] >='2019-07-20T07:08:46Z') & (df['Timestamp'] < '2019-07-20T07:10:311Z')),'Label']='Attack'
#Attack on LIT301
df.loc[((df['Timestamp'] >='2019-07-20T07:15:00Z') & (df['Timestamp'] < '2019-07-20T07:19:321Z')),'Label']='Attack'
#Attack on P601
df.loc[((df['Timestamp'] >='2019-07-20T07:26:57Z') & (df['Timestamp'] < '2019-07-20T07:30:481Z')),'Label']='Attack'
#Multipoint Attack
df.loc[((df['Timestamp'] >='2019-07-20T07:38:50Z') & (df['Timestamp'] < '2019-07-20T07:46:201Z')),'Label']='Attack'
##Attack on MV501
df.loc[((df['Timestamp'] >='2019-07-20T07:54:00Z') & (df['Timestamp'] < '2019-07-20T07:56:01Z')),'Label']='Attack'
#Attack on P301
df.loc[((df['Timestamp'] >='2019-07-20T08:02:56Z') & (df['Timestamp'] < '2019-07-20T08:16:181Z')),'Label']='Attack'


In [35]:
df

,Timestamp,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status,Label
0,2019-07-20T04:30:00Z,0.000000,729.865800,1,3,2,1,142.527557,9.293002,198.077423,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
1,2019-07-20T04:30:01Z,0.000000,729.434000,1,3,2,1,142.527557,9.293002,198.385025,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
2,2019-07-20T04:30:02.004013Z,0.000000,729.120000,1,3,2,1,142.527557,9.293002,198.436300,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
3,2019-07-20T04:30:03.004013Z,0.000000,728.688200,1,3,2,1,142.527557,9.289157,198.667000,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
4,2019-07-20T04:30:04Z,0.000000,727.706900,1,3,2,1,142.527557,9.289157,198.897720,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14991,2019-07-20T08:39:55.001007Z,4.200429,491.169769,2,2,2,1,131.408615,9.319918,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
14992,2019-07-20T08:39:56.0050048Z,4.253915,491.405273,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
14993,2019-07-20T08:39:57.0050048Z,4.303558,492.308100,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
14994,2019-07-20T08:39:58.0050048Z,4.323736,492.465100,2,2,2,1,131.408615,9.316713,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal


In [36]:
df.describe()

,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,FIT 201,...,P501 Status,P502 Status,PIT 501,PIT 502,PIT 503,FIT 601,P6 STATE,P601 Status,P602 Status,P603 Status
count,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.0,14996.000000,14996.000000,14996.000000,14996.000000,...,14996.0,14996.0,14996.000000,14996.000000,14996.000000,14996.000000,14996.0,14996.000000,14996.0,14996.0
mean,0.740876,733.960251,1.156175,2.061616,1.374166,1.0,138.766501,9.210022,247.985162,0.869760,...,2.0,1.0,160.706744,4.673115,115.157048,0.000402,2.0,1.015337,1.0,1.0
std,1.634632,110.960185,0.384272,0.240466,0.483923,0.0,8.265845,0.175812,11.806186,1.121283,...,0.0,0.0,5.707502,18.183883,5.324942,0.003317,0.0,0.122895,0.0,0.0
min,0.000000,491.169769,0.000000,2.000000,1.000000,1.0,113.849014,8.768457,198.077423,0.000000,...,2.0,1.0,158.132523,2.450902,111.654060,0.000000,2.0,1.000000,1.0,1.0
25%,0.000000,640.595184,1.000000,2.000000,1.000000,1.0,131.536789,9.090170,239.887200,0.000384,...,2.0,1.0,159.526400,2.851376,114.233528,0.000256,2.0,1.000000,1.0,1.0
50%,0.000000,819.636841,1.000000,2.000000,1.000000,1.0,143.713150,9.233082,246.218918,0.000513,...,2.0,1.0,160.231354,2.883414,114.714172,0.000320,2.0,1.000000,1.0,1.0
75%,0.000000,820.971436,1.000000,2.000000,2.000000,1.0,144.033585,9.345873,257.190460,2.320187,...,2.0,1.0,160.695969,2.963509,115.002563,0.000320,2.0,1.000000,1.0,1.0
max,4.403484,825.092957,2.000000,3.000000,2.000000,1.0,146.821335,9.490067,272.289154,2.342357,...,2.0,1.0,219.014359,192.371765,170.565247,0.137315,2.0,2.000000,1.0,1.0


In [37]:
df.columns

Index(['Timestamp', 'FIT 101', 'LIT 101', 'MV 101', 'P1_STATE', 'P101 Status',
       'P102 Status', 'AIT 201', 'AIT 202', 'AIT 203', 'FIT 201', 'LS 201',
       'LS 202', 'LSL 203', 'LSLL 203', 'MV201', 'P2_STATE', 'P201 Status',
       'P202 Status', 'P203 Status', 'P204 Status', 'P205 Status',
       'P206 Status', 'P207 Status', 'P208 Status', 'AIT 301', 'AIT 302',
       'AIT 303', 'DPIT 301', 'FIT 301', 'LIT 301', 'MV 301', 'MV 302',
       'MV 303', 'MV 304', 'P3_STATE', 'P301 Status', 'P302 Status', 'AIT 401',
       'AIT 402', 'FIT 401', 'LIT 401', 'LS 401', 'P4_STATE', 'P401 Status',
       'P402 Status', 'P403 Status', 'P404 Status', 'UV401', 'AIT 501',
       'AIT 502', 'AIT 503', 'AIT 504', 'FIT 501', 'FIT 502', 'FIT 503',
       'FIT 504', 'MV 501', 'MV 502', 'MV 503', 'MV 504', 'P5_STATE',
       'P501 Status', 'P502 Status', 'PIT 501', 'PIT 502', 'PIT 503',
       'FIT 601', 'LSH 601', 'LSH 602', 'LSH 603', 'LSL 601', 'LSL 602',
       'LSL 603', 'P6 STATE', 'P601 Statu

In [38]:
df['Label'].value_counts()

Normal    13016
Attack     1980
Name: Label, dtype: int64

In [39]:
test_df = df.drop(['Timestamp'],axis=1)

In [40]:
# Get list of categorical variables
s = (test_df.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['LS 201', 'LS 202', 'LSL 203', 'LSLL 203', 'LS 401', 'LSH 601', 'LSH 602', 'LSH 603', 'LSL 601', 'LSL 602', 'LSL 603', 'Label']


In [41]:
from sklearn.preprocessing import LabelEncoder

label_df = test_df.copy()

# inactive = 0 active = 1     label: normal = 1 , attack = 0
label_encoder = LabelEncoder()
for col in object_cols:
    label_df[col] = label_encoder.fit_transform(test_df[col])

label_df

,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,FIT 201,...,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status,Label
0,0.000000,729.865800,1,3,2,1,142.527557,9.293002,198.077423,2.335437,...,0,0,0,0,0,2,1,1,1,1
1,0.000000,729.434000,1,3,2,1,142.527557,9.293002,198.385025,2.335437,...,0,0,0,0,0,2,1,1,1,1
2,0.000000,729.120000,1,3,2,1,142.527557,9.293002,198.436300,2.335437,...,0,0,0,0,0,2,1,1,1,1
3,0.000000,728.688200,1,3,2,1,142.527557,9.289157,198.667000,2.335437,...,0,0,0,0,0,2,1,1,1,1
4,0.000000,727.706900,1,3,2,1,142.527557,9.289157,198.897720,2.335437,...,0,0,0,0,0,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14991,4.200429,491.169769,2,2,2,1,131.408615,9.319918,257.703156,2.316086,...,0,0,0,0,0,2,1,1,1,1
14992,4.253915,491.405273,2,2,2,1,131.408615,9.317354,257.703156,2.314292,...,0,0,0,0,0,2,1,1,1,1
14993,4.303558,492.308100,2,2,2,1,131.408615,9.317354,257.703156,2.313651,...,0,0,0,0,0,2,1,1,1,1
14994,4.323736,492.465100,2,2,2,1,131.408615,9.316713,257.703156,2.313651,...,0,0,0,0,0,2,1,1,1,1


In [42]:
Y = label_df.Label
X = label_df.drop(['Label'], axis=1)

In [43]:
values = X

In [44]:
Y.value_counts()

1    13016
0     1980
Name: Label, dtype: int64

In [45]:
#normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_normalized=scaler.fit_transform(values)
X_normalized

array([[0.        , 0.71482317, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.71353006, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.71258972, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97730752, 0.00340896, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98188965, 0.00387913, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98188965, 0.00517218, 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [46]:
from sklearn.model_selection import train_test_split
# # Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_normalized, Y, train_size=0.9, test_size=0.1,
                                                                random_state=0, shuffle=False)

In [47]:
X_train_full.shape

(13496, 77)

In [48]:
#function to convert to time domain dataset
def create_dataset(X, time_steps):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

In [49]:
# reshape to [samples, time_steps, n_features]

TIME_STEPS = 1
X_train = pd.DataFrame(X_train_full)
X_train = create_dataset(X_train, TIME_STEPS)
y_train = pd.DataFrame(y_train)
y_train = create_dataset(y_train, TIME_STEPS)
X_valid = pd.DataFrame(X_valid_full)
X_valid = create_dataset(X_valid, TIME_STEPS)
y_valid = pd.DataFrame(y_valid)
y_valid = create_dataset(y_valid, TIME_STEPS)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(13495, 1, 77)
(13495, 1, 1)
(1499, 1, 77)
(1499, 1, 1)


# Construct Model

In [61]:
class LSTM(nn.Module):
    def __init__(self,input_size, output_size, hidden_size, num_layers):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        #layer 1
        self.lstm = nn.LSTM(self.input_size, 
                            self.hidden_size,
                            self.num_layers,
                            batch_first=False,dropout = 0)

        self.dropout = nn.Dropout(p=0.3)
        #layer 2
        self.fc = nn.Linear(hidden_size,output_size)
      
        self.sigmoid = nn.Sigmoid()
     
    def forward(self, x):     
        per_out=[]
        lstm_out, self.hidden_cell = self.lstm(x)
        per_out.append(lstm_out)
        lstm_out = self.dropout(lstm_out)    
        out = self.fc(lstm_out)
        per_out.append(out)
        score = self.sigmoid(out)
        return score, per_out

In [51]:
def evaluate_accuracy(x,y,model):
    output,pre_out = model(x)
    output = torch.reshape(output,[-1,1])
    correct = (output.ge(0.5) == y).sum().item()
    n = y.shape[0]
    return correct/n

In [52]:
input_size = 77
output_size = 1
hidden_size = 20
num_layers =2

device = torch.device('cuda')
model = LSTM(input_size,output_size, hidden_size, num_layers).to(device)
loss_function = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [53]:
def getDataloader(X_train_full,y_train,X_valid_full,y_valid):
    TIME_STEPS = 1
    X_train = pd.DataFrame(X_train_full)
    X_train = create_dataset(X_train, TIME_STEPS)
    y_train = pd.DataFrame(y_train)
    y_train = create_dataset(y_train, TIME_STEPS)
    X_valid = pd.DataFrame(X_valid_full)
    X_valid = create_dataset(X_valid, TIME_STEPS)
    y_valid = pd.DataFrame(y_valid)
    y_valid = create_dataset(y_valid, TIME_STEPS)
# 将输入和输出封装进Data.TensorDataset()类对象
    x = torch.tensor(X_train[:,:,:]).float()
    x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    y = y_train[:]
    y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
    y = torch.tensor(y)
    y = y.float()
    torch_dataset = Data.TensorDataset(x,y)
    train_dataloader = torch.utils.data.DataLoader(torch_dataset, batch_size=60, shuffle=False, num_workers=0)
    return X_train, y_train, X_valid, y_valid, train_dataloader

In [54]:
import torch.utils.data as Data
# 将输入和输出封装进Data.TensorDataset()类对象
x = torch.tensor(X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
y = torch.tensor(y)
y = y.float()
print(x.shape)
print(y.shape)

torch_dataset = Data.TensorDataset(x,y)

torch.Size([13495, 1, 77])
torch.Size([13495, 1, 1])


In [55]:
BATCH_SIZE = 60
train_dataloader = torch.utils.data.DataLoader(torch_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [56]:
train_dataloader.batch_size

60

In [57]:
batch_array = np.zeros(len(train_dataloader))
batch_index_array = np.zeros(len(train_dataloader))

# Training and Evaluation

In [62]:
import time

time_start=time.time()

epochs = 20

for epoch in range(epochs):
    model.train()
    for batch, (batch_x, batch_y) in enumerate(train_dataloader):   
        batch_x = batch_x.cuda()   
        output,pre_out = model(batch_x)
        output = torch.reshape(output,[-1,1])

        batch_y = np.array(batch_y)
        batch_y = torch.tensor(np.reshape(batch_y,[-1,1]))
        batch_y = batch_y.float()
        batch_y = batch_y.cuda()

        loss = loss_function(output,batch_y)
        acc = evaluate_accuracy(batch_x,batch_y,model)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_array[batch] = loss.item()
        batch_index_array[batch] = batch
        if epoch % 5 == 0:
            print("epoch:{} batch:{} loss:{} acc:{}".format(epoch,batch,loss.item(),acc))

time_end=time.time()
print('totally cost',time_end-time_start)

epoch:0 batch:0 loss:0.04436497390270233 acc:1.0
epoch:0 batch:1 loss:0.04692162200808525 acc:1.0
epoch:0 batch:2 loss:0.03874116390943527 acc:1.0
epoch:0 batch:3 loss:0.04352283477783203 acc:1.0
epoch:0 batch:4 loss:0.045079655945301056 acc:1.0
epoch:0 batch:5 loss:0.04476911947131157 acc:1.0
epoch:0 batch:6 loss:0.054479531943798065 acc:1.0
epoch:0 batch:7 loss:0.03830759599804878 acc:1.0
epoch:0 batch:8 loss:0.04475592076778412 acc:1.0
epoch:0 batch:9 loss:0.04085180163383484 acc:1.0
epoch:0 batch:10 loss:0.04259198158979416 acc:1.0
epoch:0 batch:11 loss:0.04790929704904556 acc:1.0
epoch:0 batch:12 loss:0.0399688184261322 acc:1.0
epoch:0 batch:13 loss:0.04109927639365196 acc:1.0
epoch:0 batch:14 loss:0.043490130454301834 acc:1.0
epoch:0 batch:15 loss:0.04520556703209877 acc:1.0
epoch:0 batch:16 loss:0.04810512438416481 acc:1.0
epoch:0 batch:17 loss:0.04341646283864975 acc:1.0
epoch:0 batch:18 loss:0.05030884966254234 acc:1.0
epoch:0 batch:19 loss:0.04468575492501259 acc:1.0
epoch:0 

epoch:0 batch:165 loss:3.4479479789733887 acc:0.0
epoch:0 batch:166 loss:2.128584384918213 acc:0.0
epoch:0 batch:167 loss:1.0094629526138306 acc:0.38333333333333336
epoch:0 batch:168 loss:1.9934817552566528 acc:0.0
epoch:0 batch:169 loss:0.8557331562042236 acc:0.48333333333333334
epoch:0 batch:170 loss:1.113670825958252 acc:0.1
epoch:0 batch:171 loss:1.3217976093292236 acc:0.06666666666666667
epoch:0 batch:172 loss:1.3754130601882935 acc:0.08333333333333333
epoch:0 batch:173 loss:1.2440848350524902 acc:0.06666666666666667
epoch:0 batch:174 loss:1.0962738990783691 acc:0.05
epoch:0 batch:175 loss:1.7937302589416504 acc:0.06666666666666667
epoch:0 batch:176 loss:1.7577645778656006 acc:0.1
epoch:0 batch:177 loss:0.17321926355361938 acc:1.0
epoch:0 batch:178 loss:0.18668527901172638 acc:1.0
epoch:0 batch:179 loss:0.23137891292572021 acc:1.0
epoch:0 batch:180 loss:0.6232397556304932 acc:0.7333333333333333
epoch:0 batch:181 loss:1.7124347686767578 acc:0.0
epoch:0 batch:182 loss:1.646007418632

epoch:5 batch:103 loss:0.020581932738423347 acc:1.0
epoch:5 batch:104 loss:0.0168137326836586 acc:1.0
epoch:5 batch:105 loss:0.02125728502869606 acc:1.0
epoch:5 batch:106 loss:0.020029447972774506 acc:1.0
epoch:5 batch:107 loss:0.019032305106520653 acc:1.0
epoch:5 batch:108 loss:0.020784849300980568 acc:1.0
epoch:5 batch:109 loss:0.01696639321744442 acc:1.0
epoch:5 batch:110 loss:0.018230248242616653 acc:1.0
epoch:5 batch:111 loss:0.019445622339844704 acc:1.0
epoch:5 batch:112 loss:0.057797499001026154 acc:1.0
epoch:5 batch:113 loss:0.07462543249130249 acc:1.0
epoch:5 batch:114 loss:0.07209409773349762 acc:1.0
epoch:5 batch:115 loss:0.07575671374797821 acc:1.0
epoch:5 batch:116 loss:0.06120050325989723 acc:1.0
epoch:5 batch:117 loss:0.06134217977523804 acc:1.0
epoch:5 batch:118 loss:0.06808482855558395 acc:1.0
epoch:5 batch:119 loss:0.06441875547170639 acc:1.0
epoch:5 batch:120 loss:0.06349940598011017 acc:1.0
epoch:5 batch:121 loss:0.057994991540908813 acc:1.0
epoch:5 batch:122 loss:0

epoch:10 batch:32 loss:0.09852023422718048 acc:1.0
epoch:10 batch:33 loss:0.08019188046455383 acc:1.0
epoch:10 batch:34 loss:0.0826493576169014 acc:1.0
epoch:10 batch:35 loss:0.07875112444162369 acc:1.0
epoch:10 batch:36 loss:0.0742906704545021 acc:1.0
epoch:10 batch:37 loss:0.07848785817623138 acc:1.0
epoch:10 batch:38 loss:0.08571828156709671 acc:1.0
epoch:10 batch:39 loss:0.08352938294410706 acc:1.0
epoch:10 batch:40 loss:0.08660920709371567 acc:1.0
epoch:10 batch:41 loss:0.085704505443573 acc:1.0
epoch:10 batch:42 loss:0.0798739492893219 acc:1.0
epoch:10 batch:43 loss:0.07673852145671844 acc:1.0
epoch:10 batch:44 loss:0.04420093074440956 acc:1.0
epoch:10 batch:45 loss:0.014214790426194668 acc:1.0
epoch:10 batch:46 loss:0.010610450059175491 acc:1.0
epoch:10 batch:47 loss:0.015165833756327629 acc:1.0
epoch:10 batch:48 loss:0.011932946741580963 acc:1.0
epoch:10 batch:49 loss:0.012828364968299866 acc:1.0
epoch:10 batch:50 loss:0.013858728110790253 acc:1.0
epoch:10 batch:51 loss:0.01171

epoch:10 batch:195 loss:0.8751670122146606 acc:0.05
epoch:10 batch:196 loss:0.7328516244888306 acc:0.2833333333333333
epoch:10 batch:197 loss:0.8028134703636169 acc:0.08333333333333333
epoch:10 batch:198 loss:0.4358341097831726 acc:0.65
epoch:10 batch:199 loss:0.063764289021492 acc:1.0
epoch:10 batch:200 loss:0.05303521454334259 acc:1.0
epoch:10 batch:201 loss:0.06084617227315903 acc:1.0
epoch:10 batch:202 loss:0.6633002758026123 acc:0.4166666666666667
epoch:10 batch:203 loss:0.8802027702331543 acc:0.38333333333333336
epoch:10 batch:204 loss:0.37722933292388916 acc:0.9833333333333333
epoch:10 batch:205 loss:0.29052916169166565 acc:0.9666666666666667
epoch:10 batch:206 loss:0.13476306200027466 acc:1.0
epoch:10 batch:207 loss:0.1354053020477295 acc:1.0
epoch:10 batch:208 loss:0.12804214656352997 acc:1.0
epoch:10 batch:209 loss:0.12324175238609314 acc:1.0
epoch:10 batch:210 loss:0.1322479248046875 acc:1.0
epoch:10 batch:211 loss:1.0791367292404175 acc:0.5833333333333334
epoch:10 batch:212

epoch:15 batch:130 loss:0.06242457032203674 acc:1.0
epoch:15 batch:131 loss:0.06200059875845909 acc:1.0
epoch:15 batch:132 loss:0.06316976249217987 acc:1.0
epoch:15 batch:133 loss:0.062063466757535934 acc:1.0
epoch:15 batch:134 loss:0.010762871243059635 acc:1.0
epoch:15 batch:135 loss:0.010900722816586494 acc:1.0
epoch:15 batch:136 loss:0.012194213457405567 acc:1.0
epoch:15 batch:137 loss:0.011287415400147438 acc:1.0
epoch:15 batch:138 loss:0.01258205994963646 acc:1.0
epoch:15 batch:139 loss:0.010901475325226784 acc:1.0
epoch:15 batch:140 loss:0.011505058035254478 acc:1.0
epoch:15 batch:141 loss:0.009268355555832386 acc:1.0
epoch:15 batch:142 loss:0.013142057694494724 acc:1.0
epoch:15 batch:143 loss:0.010014881379902363 acc:1.0
epoch:15 batch:144 loss:0.010327554307878017 acc:1.0
epoch:15 batch:145 loss:0.013592967763543129 acc:1.0
epoch:15 batch:146 loss:0.009442622773349285 acc:1.0
epoch:15 batch:147 loss:0.01301547046750784 acc:1.0
epoch:15 batch:148 loss:0.009677483700215816 acc:1.

In [63]:
model.eval()
x = torch.tensor( X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
y = torch.tensor(np.reshape(y,[-1,1]))
y = y.float()
    
x = x.cuda()
y = y.cuda()

x_valid = torch.tensor(X_valid[:,:,:]).float()
x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
y_val = y_valid[:]
y_val = np.array(y_val)
y_val = torch.tensor(np.reshape(y_val,[-1,1]))
y_val = y_val.float()

x_valid = x_valid.cuda()
y_val = y_val.cuda()

print("epoch:{} loss:{}".format(epoch,loss.item()))
print("train acc", evaluate_accuracy(x,y,model))
print("valid acc:",evaluate_accuracy(x_valid,y_val,model))

epoch:19 loss:0.07267970591783524
train acc 0.9190070396443127
valid acc: 0.052701801200800535


In [64]:
import sklearn.metrics as sm
output,pre_out = model(x_valid)
output = torch.reshape(output,[-1,1])
output = output.ge(0.5)

output_cpu = output.cpu()
cr = sm.classification_report(y_valid[:,0], output_cpu)
print(cr)

              precision    recall  f1-score   support

           0       0.05      1.00      0.10        79
           1       0.00      0.00      0.00      1420

    accuracy                           0.05      1499
   macro avg       0.03      0.50      0.05      1499
weighted avg       0.00      0.05      0.01      1499



D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
